In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import re
import os

In [12]:
os.makedirs("./data/wiki40b-txt/")

In [8]:
lang_code = "en"
r1 = "_START_ARTICLE_\n[^_]*"
r2 = "_START_PARAGRAPH_\n"
r3 = "_START_SECTION_\n[^_]*"
r4 = "_NEWLINE_"

REGEX = re.compile(f"({r1}|{r2}|{r3}|{r4})")

## Generating txt files from the Wiki-40b dataset

In [9]:
def process_tf_dataset(ds, num_tokens, output_file):
    # Turn to a numpy df so that we can easily extract text
    # numpy_items = tfds.as_numpy(ds)
    token_count = 0

    with open(output_file, "a") as f:
        for batch in ds.as_numpy_iterator():
            # text is the feature we want to extract
            for item in batch.get("text"):
                text = item.decode("UTF-8")
                text = re.sub(REGEX, " ", text)
                text = re.sub("\s+", " ", text).strip()
                f.write(text)
                f.write("\n")
                token_count += len(text.split())
                if num_tokens > 0 and token_count > num_tokens:
                    break

In [20]:
# load from validation split of the wiki-40b dataset, which has 163597 entries.
# Don't run repeatedly once you loaded.
ds = tfds.load(
    f"wiki40b/{lang_code}",
    split="train",
    shuffle_files=True,
    data_dir="./data/",
    batch_size=128,
)

In [21]:
# generating pure txt file for train split in wiki-40b
process_tf_dataset(
    ds, -1, "./data/wiki40b-txt/" + lang_code + ".train"
)

2023-07-18 23:14:18.518598: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [128]
	 [[{{node Placeholder/_0}}]]
2023-07-18 23:14:18.519921: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [128]
	 [[{{node Placeholder/_2}}]]


In [18]:
# don't know why changing the split matters
ds = tfds.load(
    f"wiki40b/{lang_code}",
    split="test",
    shuffle_files=True,
    data_dir="./data/",
    batch_size=128,
)

In [ ]:
process_tf_dataset(
    ds, -1, "./data/wiki40b-txt/" + lang_code + ".test"
)

In [ ]:
ds = tfds.load(
    f"wiki40b/{lang_code}",
    split="validation",
    shuffle_files=True,
    data_dir="./data/",
    batch_size=128,
)

In [ ]:
process_tf_dataset(
    ds, -1, "./data/wiki40b-txt/" + lang_code + ".validation"
)
# why this has so many entries?
# 3 and 300 have same results, but -1 a lot more.

In [22]:
dirname = os.path.dirname("wiki40b-txt/")

In [23]:
dirname

'wiki40b-txt'

In [ ]:
Ichijō Fuyuyoshi (一条 冬良, July 29, 1465 – April 21, 1514), son of regent Kaneyoshi, was a kugyō or court noble of the Muromachi period (1336–1573) of Japan. He held a regent position kampaku two times from 1488 to 1493 and from 1497 to 1501. He adopted Fusamichi as son who was also his daughter's husband.

## Testing training a BPETokenizer from scratch on different languages

In [1]:
import argparse
import os

from tokenizers import AddedToken
from tokenizers.implementations import ByteLevelBPETokenizer
from transformers import AutoTokenizer

/Users/sally/Desktop/Thesis_readings/Typological_Universal_BabyLM/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sample = {
    "en": "Hello, y'all! How are you 😁? (just testing the tokenizer)",
    "ja": "おやすみなさい",
    "it": "Stiamo cercando una gioielleria.",
    "ja_en": "おやすみなさい"
             "Hello, y'all! How are you 😁? (just testing the tokenizer)",
    "it_en": "Stiamo cercando una gioielleria."
             "Hello, y'all! How are you 😁? (just testing the tokenizer)",
}

In [4]:
def train_tokenizer():

    bpe_tokenizer = ByteLevelBPETokenizer()

    files = [f"./data/wiki40b-txt/en.small"]

    bpe_tokenizer.train(files=files, vocab_size=32000, min_frequency=2)

    tokenizer_path = f'./data/tokenizer/en'
    if not os.path.exists(tokenizer_path):
        os.makedirs(tokenizer_path)

    # save the vocab.json and merges.txt files of the trained bpe tokenizer
    bpe_tokenizer.save_model(tokenizer_path)

    model_tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, tokenizer_type="gpt2")
    model_tokenizer.model_max_length = 512
    model_tokenizer.add_special_tokens({"pad_token": AddedToken("<pad>", normalized=True)})

    print(f'Tokenizer vocab size: {len(model_tokenizer)}')
    print(f'Tokenizer max sequence length: {model_tokenizer.model_max_length} \n')

    # save the full model tokenizer configuration files
    model_tokenizer.save_pretrained(tokenizer_path)

    output = model_tokenizer.encode_plus(sample["en"])
    print(output.tokens(), '\n')

In [5]:
train_tokenizer()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Tokenizer vocab size: 32002
Tokenizer max sequence length: 512 

['H', 'ello', ',', 'Ġy', "'", 'all', '!', 'ĠHow', 'Ġare', 'Ġyou', 'Ġ', 'ð', 'Ł', 'ĺ', 'ģ', '?', 'Ġ(', 'just', 'Ġtesting', 'Ġthe', 'Ġto', 'ken', 'izer', ')'] 

